In [328]:
from datetime import time
import webbrowser
import os

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc, html
import dash_mantine_components as dmc
import geopandas as gpd

from DataInterface.TrafficDataInterface import CityTrafficData, TrafficType
from DataInterface.GeoDataInterface import GeoData, GeoDataType
from DataInterface.AdminDataInterface import AdminData
from Utils import City
from FeatureExtraction.SessionDistribution.SessionDistributionCalculator import SessionDistributionCalculator
from FeatureExtraction.IrisFeatureCalculator import IrisFeatureCalculator
from FeatureSelection.Regression import Regression

In [3]:
%load_ext autoreload
%autoreload 2

In [56]:
traffic_data = CityTrafficData(city=City.LYON, geo_data_type=GeoDataType.IRIS)

100%|██████████| 77/77 [01:12<00:00,  1.07it/s]


In [139]:
geo_data = GeoData()
geo_data.load(GeoDataType.IRIS)
geo_data.load(GeoDataType.POLLING_STATION)

In [108]:
election_data = ElectionData()

In [109]:
admin_data = AdminData()

In [57]:
service_consumption_by_location = traffic_data.get_service_consumption_by_location(start=time(18,30), end=time(23))

In [58]:
feature_calculator = ServiceConsumptionFeatureCalculator(service_consumption_by_location=service_consumption_by_location)

In [333]:
service_diversity = feature_calculator.get_consumption_diversity()

In [334]:
geo_data_iris = geo_data.get_geo_data(geometry=GeoDataType.IRIS, subset=service_diversity.index)

In [239]:
expectations_with_geo_data = gpd.GeoDataFrame(service_diversity.merge(geo_data_iris.set_index(GeoDataType.IRIS.value), left_index=True, right_index=True))[['geometry', 'entropy']]
html_map = expectations_with_geo_data.explore(column='entropy', legend=True, tiles="CartoDB positron", cmap='plasma')
html_map.save('temp/diversity_map.html')
webbrowser.open('file://' + os.path.realpath('temp/diversity_map.html'))

True

In [335]:
iris_subset = service_diversity.index

In [336]:
feature_constructor = FeatureConstructor(admin_data=admin_data, geo_data=geo_data)

In [337]:
iris_pop_density = feature_constructor.iris_density(iris=iris_subset, var_names=['P19_POP']).rename(columns={'P19_POP': 'POP_DEN'})

In [338]:
entropy = service_diversity['entropy'].to_frame()

In [339]:
data = pd.merge(iris_pop_density, entropy, left_index=True, right_index=True, how='inner')
data = np.log(data)
data.dropna(inplace=True)
data.shape, np.array(data.columns)

((2735, 2), array(['POP_DEN', 'entropy'], dtype=object))

In [340]:
x_axis = 'POP_DEN'
features = data[[c for c in data.columns if c not in ['entropy']]]
labels = data['entropy'].to_frame()
regression = Regression(features=features, labels=labels)
regression.plot(x_axis=x_axis)

In [341]:
election_data_by_iris = election_data.get_election_data_table_iris_by_column(geo_data=geo_data, subset=traffic_data.data.coords[GeoDataType.IRIS.value].values, column='list_number', value='pct_votes_to_list_among_votes', aggregation_method='mean') / 100

In [342]:
election_feature_calculator = ElectionDataFeatureCalculator(election_result_by_location=election_data_by_iris)

In [343]:
election_diversity = election_feature_calculator.get_diversity()

In [344]:
geo_data_iris = geo_data.get_geo_data(geometry=GeoDataType.IRIS, subset=election_data_by_iris.index).set_index(GeoDataType.IRIS.value)

In [345]:
vote_entropy_with_iris_geo_data = geo_data_iris.merge(election_diversity['entropy'].to_frame(), right_index=True, left_index=True, how='inner')

In [346]:
html_map = vote_entropy_with_iris_geo_data.explore(column='entropy', legend=True, tiles="CartoDB positron", cmap='plasma')
html_map.save('temp/vote_entropy_map.html')
webbrowser.open('file://' + os.path.realpath('temp/vote_entropy_map.html'))

True

In [323]:
entropy_service = service_diversity['entropy'].to_frame(name='entropy_service')

In [347]:
entropy_vote = vote_entropy_with_iris_geo_data['entropy'].to_frame(name='entropy_vote')

In [348]:
data = pd.merge(entropy_service, entropy_vote, left_index=True, right_index=True, how='inner')

In [349]:
data.dropna(inplace=True)
data.shape

(2930, 2)

In [350]:
x_axis = 'entropy_service'
features = data['entropy_service'].to_frame()
labels = data['entropy_vote'].to_frame()
regression = Regression(features=features, labels=labels)
regression.plot(x_axis=x_axis)